In [1]:
import json
import csv

In [2]:
"""
This can probably be improved but serves the purpose of our analysis.
I used Chris's schema so it will be useful for Quarz analysis.

"""

EMOTIONS = [
    'contempt',
    'fear',
    'sadness',
    'disgust',
    'anger',
    'neutral',
    'happiness',
    'surprise',
]


CSV_COLUMNS = [
    'url',
    'frame_time',
    'trump_visible',
    'trump_happiness',
    'trump_neutral',
    'trump_anger',
    'trump_contempt',
    'trump_suprise',
    'trump_sadness',
    'trump_fear',
    'trump_disgust',
    'clinton_visible',
    'clinton_happiness',
    'clinton_neutral',
    'clinton_anger',
    'clinton_contempt',
    'clinton_suprise',
    'clinton_sadness',
    'clinton_fear',
    'clinton_disgust'
]

In [3]:
with open('/Users/Amiros/GitHub/debate/debate1_full.json') as data_file:    
    json_data = json.load(data_file)
    
json_data = {int(k):v for k,v in json_data.items()} #converting keys to integer   
frame_keys = sorted(json_data.keys())

In [4]:
csv_data = []
   
for key in frame_keys:
    url = 'https://s3-us-west-2.amazonaws.com/debateinemotion/debate1/frame%s.jpg' % key
    frame = json_data[key]
        
    row = {
        'frame': frame,
        'url': url,
        'frame_time': key,
        'trump_visible': False,
        'clinton_visible': False
    }
    
    if len(frame) > 2:
        print('Frame %s has more than 2 faces (%s)' % (key, url))
        continue
    elif len(frame) < 2:
        for face in frame:
            if face['faceAttributes']['gender'] == 'male':
                prefix = 'trump_'
                row[prefix + 'visible'] = True
                for emotion in EMOTIONS:
                    #print(face['scores'][emotion])
                    row[prefix + emotion] = face['scores'][emotion]
                ### assign emotions vale to each candidate and change the name to the respective one 
                    
            if face['faceAttributes']['gender'] == 'female':
                prefix = 'clinton_'
                row[prefix + 'visible'] = True
                for emotion in EMOTIONS:
                    #print(face['scores'][emotion])
                    row[prefix + emotion] = face['scores'][emotion]
            row.pop("frame", None)        
      
    elif len(frame) == 2:
        row['trump_visible'] = True
        row['clinton_visible'] = True
        for face in frame:
            if face['faceAttributes']['gender'] == 'male':
                prefix = 'trump_'
                row[prefix + 'visible'] = True
                for emotion in EMOTIONS:
                    #print(face['scores'][emotion])
                    row[prefix + emotion] = face['scores'][emotion]
                ### assign emotions vale to each candidate and change the name to the respective one 
                    
            if face['faceAttributes']['gender'] == 'female':
                prefix = 'clinton_'
                row[prefix + 'visible'] = True
                for emotion in EMOTIONS:
                    #print(face['scores'][emotion])
                    row[prefix + emotion] = face['scores'][emotion]
        row.pop("frame", None)        

    csv_data.append(row)

In [6]:
import pandas as pd
df = pd.DataFrame(csv_data)

In [10]:
df.to_csv('debate1_v2.csv', sep=',')